## 📝 Free/Freemium API Provider Comparison

| Provider        |  Free Tier Rate Limits | Data Freshness | Historical Depth | Python SDK | 1-Min Support | Notes |
|-----------------|------------------------|----------------|------------------|-------------|---------------|-------|

Data Coverage: How far back does the historical data go? Is it up-to-date?
Data Fields: Does it provide OHLC, adjusted close, and volume?
Rate Limits: Free-tier request limits per minute/day.
Ease of Use: Python SDK availability, ease of authentication.
Docs & Support: Quality of documentation and community examples.
Reliability: Are there known issues like downtime or staleness?


| Alpha Vantage   | 5/min, 500/day         | ✅ Often delayed | Medium          | ✅           | ❌ (free)      |       |
| Tiingo          | ~500 symbols/day       | ✅ Fresh        | ✅ Deep          | ✅           | ❌ (free)      |       |
| ...             | ...                    | ...            | ...              | ...         | ...           |       |

### ✅ API Shortlist
| Provider              |  Pros                                                        | Cons                                                         |
|-----------------------|--------------------------------------------------------------|--------------------------------------------------------------|

e.g. 1. Alpha Vantage
Pros: Free API key; Good documentation; Simple alphavantage Python SDK.
Cons: 5 calls/min, 500 calls/day (free tier); Delays in data refresh; No intraday data on free tier.
e.g. 2. Tiingo
Pros: Free tier includes end-of-day data; Good coverage, adjusted data available; tiingo Python client available.
Cons: Requires registration and API key; Limited to ~500 daily symbols.

Based on the evaluation, .... is the most suitable candidate for production use......

In [ ]:
# 🚀 Import and Setup
import pandas as pd
import matplotlib.pyplot as plt  # or plotly
from main import fetch_daily_data

In [ ]:
# 📦 Try fetching from each API
df = fetch_daily_data("AAPL", "2023-01-01", "2023-12-31", source="tiingo")
df.head()

In [ ]:
# 📈 Visualizations
# Show OHLC candlestick for a ticker.
# Plot volume alongside price.
df.set_index("date")[["close"]].plot(title="AAPL Daily Close")

## 🔍 Bonus: Intraday (1-minute) Data Exploration

- Which APIs support it?
- How to access?
- Code snippets (optional)